In [8]:
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
import re

# Define our state
class State(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], list.append]
    payee: str
    amount: float
    current_step: str

# Initialize our LLM
llm = ChatOllama(model="phi:3.5", temperature=0)

# Define our nodes
def collect_info(state: State) -> State:
    messages = state['messages']
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a banking assistant. Collect the payee name and payment amount from the user. Ask for one piece of information at a time."),
        ("human", "I want to make a payment."),
        *messages
    ])
    response = llm.invoke(prompt.format())
    state['messages'].append(AIMessage(content=response))
    return state

def validate_info(state: State) -> State:
    messages = state['messages']
    last_message = messages[-2].content if len(messages) >= 2 else ""
    ai_response = messages[-1].content

    # Check for payee
    if state['payee'] == "" and "name" in last_message.lower():
        payee_match = re.search(r'\b[A-Z][a-z]+ [A-Z][a-z]+\b', last_message)
        if payee_match:
            state['payee'] = payee_match.group()
        else:
            state['messages'].append(AIMessage(content="I'm sorry, I couldn't identify a valid name. Could you please provide the payee's full name (First Last)?"))
    
    # Check for amount
    if state['amount'] == 0 and "amount" in last_message.lower():
        amount_match = re.search(r'\$?(\d+(\.\d{2})?)', last_message)
        if amount_match:
            state['amount'] = float(amount_match.group(1))
        else:
            state['messages'].append(AIMessage(content="I'm sorry, I couldn't identify a valid amount. Could you please provide the amount in dollars (e.g., 50.00)?"))

    # If both are collected, confirm
    if state['payee'] != "" and state['amount'] != 0:
        confirm_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a banking assistant. Confirm the payment details with the user."),
            ("human", f"Please confirm: Pay {state['payee']} the amount of ${state['amount']:.2f}. Is this correct?")
        ])
        response = llm.invoke(confirm_prompt.format())
        state['messages'].append(AIMessage(content=response))
    
    return state

def process_payment(state: State) -> State:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a banking assistant. Process the payment and inform the user."),
        ("human", f"Process payment of ${state['amount']:.2f} to {state['payee']}."),
    ])
    response = llm.invoke(prompt.format())
    state['messages'].append(AIMessage(content=response))
    return state

# Define our edges
def should_continue_collecting(state: State):
    if state['payee'] == "" or state['amount'] == 0:
        return "collect_info"
    else:
        return "validate_info"

def should_process_payment(state: State):
    last_message = state['messages'][-2].content.lower()
    if "yes" in last_message or "correct" in last_message:
        return "process_payment"
    elif "no" in last_message or "incorrect" in last_message:
        state['payee'] = ""
        state['amount'] = 0
        return "collect_info"
    else:
        return "validate_info"

def should_end(state: State):
    return END

# Create our graph
workflow = StateGraph(State)

# Add nodes
workflow.add_node("collect_info", collect_info)
workflow.add_node("validate_info", validate_info)
workflow.add_node("process_payment", process_payment)

# Add edges
workflow.add_edge("collect_info", should_continue_collecting)
workflow.add_edge("validate_info", should_continue_collecting)
workflow.add_edge("validate_info", should_process_payment)
workflow.add_edge("process_payment", should_end)

# Set entry point
workflow.set_entry_point("collect_info")

# Compile the graph
app = workflow.compile()

display(Image(workflow.get_graph(xray=1).draw_mermaid_png()))


ValueError: Node `validate_info` is not reachable

In [ ]:

# Example usage
initial_state = {
    "messages": [],
    "payee": "",
    "amount": 0,
    "current_step": "collect_info"
}

for output in app.stream(initial_state):
    if '__end__' not in output:
        response = output['messages'][-1].content
        print(f"Chatbot: {response}")
        user_input = input("User: ")
        output['messages'].append(HumanMessage(content=user_input))
    else:
        print("Transaction completed.")
        break

In [11]:
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
import re

# Define our state
class State(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], list.append]
    payee: str
    amount: float
    confirmed: bool

# Initialize our LLM
llm = ChatOllama(model="phi:3.5", temperature=0)

# Define our nodes
def collect_info(state: State) -> State:
    messages = state['messages']
    
    if state['payee'] == "":
        prompt = "Please provide the payee's full name."
    elif state['amount'] == 0:
        prompt = f"Please provide the amount to pay {state['payee']}."
    else:
        prompt = f"Please confirm: Pay {state['payee']} the amount of ${state['amount']:.2f}. Is this correct? (Yes/No)"

    llm_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a banking assistant. Collect payment information from the user."),
        ("human", prompt),
        *messages
    ])
    response = llm.invoke(llm_prompt.format())
    state['messages'].append(AIMessage(content=response))
    return state

def validate_info(state: State) -> State:
    if len(state['messages']) > 0:
        user_message = state['messages'][-2].content
        if state['payee'] == "":
            payee_match = re.search(r'\b[A-Z][a-z]+ [A-Z][a-z]+\b', user_message)
            if payee_match:
                state['payee'] = payee_match.group()
        elif state['amount'] == 0:
            amount_match = re.search(r'\$?(\d+(\.\d{2})?)', user_message)
            if amount_match:
                state['amount'] = float(amount_match.group(1))
        elif not state['confirmed']:
            if "yes" in user_message.lower():
                state['confirmed'] = True
            elif "no" in user_message.lower():
                state['payee'] = ""
                state['amount'] = 0
    return state

def process_payment(state: State) -> State:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a banking assistant. Process the payment and inform the user."),
        ("human", f"Process payment of ${state['amount']:.2f} to {state['payee']}."),
    ])
    response = llm.invoke(prompt.format())
    state['messages'].append(AIMessage(content=response))
    return state

# Define our edges
def router(state: State):
    if not state['confirmed']:
        if state['payee'] == "" or state['amount'] == 0:
            return "collect_info"
        else:
            return "validate_info"
    else:
        return "process_payment"

# Create our graph
workflow = StateGraph(State)

# Add nodes
workflow.add_node("collect_info", collect_info)
workflow.add_node("validate_info", validate_info)
workflow.add_node("process_payment", process_payment)

# Add edges
workflow.add_edge("collect_info", "validate_info")
workflow.add_edge("validate_info", router)
workflow.add_edge("process_payment", END)

# Set entry point
workflow.set_entry_point("collect_info")

# Compile the graph
app = workflow.compile()

# Example usage
initial_state = {
    "messages": [],
    "payee": "",
    "amount": 0,
    "confirmed": False
}

for output in app.stream(initial_state):
    if '__end__' not in output:
        response = output['messages'][-1].content
        print(f"Chatbot: {response}")
        user_input = input("User: ")
        output['messages'].append(HumanMessage(content=user_input))
    else:
        print("Transaction completed.")
        break


ValueError: Node `process_payment` is not reachable